<a href="https://colab.research.google.com/github/ruslanmv/AWS-Bedrock-LLMs-in-Google-Colab/blob/master/bedrock_nova_colab_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amazon Bedrock – Simple Inference (Nova Lite) in Google Colab

This notebook performs a minimal text inference against **`amazon.nova-lite-v1:0`** on Amazon Bedrock using credentials stored in **Colab `userdata`**.

> **Before you run this notebook**  
> Go to: **Colab → Tools → User data** and add the following keys:
>
> - `AWS_ACCESS_KEY_ID`  
> - `AWS_SECRET_ACCESS_KEY`  
> - `AWS_DEFAULT_REGION` (e.g., `us-east-1`)  
> - *(optional)* `AWS_BEARER_TOKEN_BEDROCK` – if this value is actually an AWS **session token**, it will be mapped to `AWS_SESSION_TOKEN` automatically below.
>
> If you are using long‑lived IAM user credentials, you typically **won't** need a session token. If you are using temporary creds (STS), you **will** need it.


In [1]:
# ==============================================================================
# 1. INSTALL LIBRARIES
# ==============================================================================
# We need the AWS SDK for Python (boto3) to communicate with Bedrock.

import sys, subprocess, importlib
def pip_install(pkg):
    if importlib.util.find_spec(pkg) is None:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ["boto3", "botocore"]:
    pip_install(pkg)

print("✔ Dependencies ready")

✔ Dependencies ready


In [2]:
# ==============================================================================
# 2. IMPORTS AND CONFIGURATION
# ==============================================================================
import boto3
import json
from google.colab import userdata

print("✅ Libraries imported.")

# Load credentials securely from Colab's userdata secrets manager.
try:
    aws_access_key_id = userdata.get('AWS_ACCESS_KEY_ID')
    aws_secret_access_key = userdata.get('AWS_SECRET_ACCESS_KEY')
    aws_default_region = userdata.get('AWS_DEFAULT_REGION')
    print("✅ AWS credentials loaded successfully.")
except Exception as e:
    print(f"🛑 Error loading credentials. Please ensure you have set them up in Colab secrets. Error: {e}")
    # Stop execution if credentials are not found
    raise SystemExit()

# ==============================================================================
# 3. SETUP BEDROCK CLIENT
# ==============================================================================
# Create a boto3 client to interact with the Bedrock Runtime.
# This client will handle the API requests to the model.
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name=aws_default_region,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)
print(f"✅ Boto3 client created for region: {aws_default_region}.")

✅ Libraries imported.
✅ AWS credentials loaded successfully.
✅ Boto3 client created for region: eu-north-1.


In [3]:
# ==============================================================================
# 4. PREPARE THE INFERENCE PAYLOAD
# ==============================================================================
# Define the model ID and the prompt for the model.
model_id = "amazon.nova-lite-v1:0"
user_prompt = "Explain the difference between a star and a planet in three simple points."

# Structure the payload according to the model's expected input format.
# The `body` contains the inference configuration and the user message.
inference_payload = {
    "inferenceConfig": {
        "max_new_tokens": 1000,
        "temperature": 0.7, # Controls randomness. Lower is more predictable.
        "top_p": 0.9        # Nucleus sampling.
    },
    "messages": [
        {
            "role": "user",
            "content": [
                {
                    "text": user_prompt
                }
            ]
        }
    ]
}

# Convert the Python dictionary to a JSON string.
body = json.dumps(inference_payload)
print("✅ Inference payload prepared.")

# ==============================================================================
# 5. INVOKE THE MODEL AND PROCESS THE RESPONSE
# ==============================================================================
try:
    print("\n🚀 Sending request to Bedrock...")
    # Invoke the model with the prepared payload.
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=body
    )
    print(response)

    # The response body is a streaming object, so we need to read and parse it.
    response_body = json.loads(response.get("body").read())

    # ❗ CORRECTED PART ❗
    # Extract the generated text from the model's response.
    # The structure for this model should be {'output': {'message': {'content': [{'text': '...'}]}}, ...}
    model_response = response_body['output']['message']['content'][0]['text']


    # --- Display the results ---
    print("\n" + "="*50)
    print("✅ INFERENCE COMPLETE")
    print("="*50)
    print(f"\n🗣️ USER PROMPT:\n{user_prompt}")
    print("\n🤖 MODEL RESPONSE:")
    print(model_response)
    print("\n" + "="*50)

except Exception as e:
    print(f"🛑 An error occurred during inference: {e}")

✅ Inference payload prepared.

🚀 Sending request to Bedrock...
{'ResponseMetadata': {'RequestId': '2ba3e8e4-0347-4dd8-a176-45f69e43918c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Fri, 03 Oct 2025 13:37:08 GMT', 'content-type': 'application/json', 'content-length': '1240', 'connection': 'keep-alive', 'x-amzn-requestid': '2ba3e8e4-0347-4dd8-a176-45f69e43918c', 'x-amzn-bedrock-invocation-latency': '1135', 'x-amzn-bedrock-cache-write-input-token-count': '0', 'x-amzn-bedrock-cache-read-input-token-count': '0', 'x-amzn-bedrock-output-token-count': '221', 'x-amzn-bedrock-input-token-count': '14'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x791cc2c8c580>}

✅ INFERENCE COMPLETE

🗣️ USER PROMPT:
Explain the difference between a star and a planet in three simple points.

🤖 MODEL RESPONSE:
Certainly! Here are three simple points to explain the difference between a star and a planet:

1. **Energy Source**:
   - **Star**: A sta